# About
This notebook is used to update `data/manual/ba_reference.csv` using the FERC allowable BA entries list, which is updated on a quarterly basis

In [ ]:
import os
import requests
import pandas as pd
import numpy as np

In [ ]:
ba_reference = pd.read_csv('../data/manual/ba_reference.csv')
ferc_bas = pd.read_csv("../data/downloads/ferc/ferc_bas.csv")

# merge the ferc data into the manual table
ba_reference_updated = ba_reference.merge(ferc_bas, how='outer', on='ba_code', indicator='source', suffixes=(None,"_ferc"))

# fill any missing data in the manual table with the data from ferc
for col in ['ba_name', 'us_ba', 'activation_date', 'retirement_date',]:
    ba_reference_updated[col] = ba_reference_updated[col].fillna(ba_reference_updated[f'{col}_ferc'])
    
    # wherever the ferc entry does not match the EIA entry, keep the ferc entry too
    ba_reference_updated.loc[(ba_reference_updated[f'{col}_ferc'] == ba_reference_updated[col]), f'{col}_ferc'] = np.NaN


ba_reference_updated['source'] = ba_reference_updated['source'].replace({'left_only':'EIA','right_only':'FERC','both':'EIA & FERC'})
# drop the us_ba_ferc column
ba_reference_updated = ba_reference_updated.drop(columns=['us_ba_ferc'])


ba_reference_updated

In [ ]:
ba_reference_updated.to_csv('../data/manual/ba_reference_updated.csv', index=False)